# Devoir 2
rédigé par Corey Ducharme et Olivier Sirois

## Préamble

In [1]:
using JuMP
using Clp

Considérons le problème d'allocation de la demande stochastique.

## Probèle avec décisions matriciel
Définissons notre problème intuitivement sous forme matricielle.

On défini nos variables
$$ x = (x_1, x_2, x_3, x_4)^{\intercal}   \\
B = (10, 7, 16, 6)^{\intercal}  \\ 
y_{mat} = \begin{pmatrix}y_{11} & y_{12} & y_{13} \\ y_{21} & y_{22} & y_{23} \\ y_{31} & y_{32} & y_{33} \\ y_{41} & y_{42} & y_{43}\end{pmatrix} \\ 
A_{mat} = \begin{pmatrix}40 & 24 & 4 \\ 45 & 27 & 4.5 \\ 32 & 19.2 & 3.2 \\ 55 & 33 & 5.5\end{pmatrix} $$

où $x_i$ représente la capacité de production de l'usine i,

B le vector des coûts de production,

$y_{ij}$ la production de glace j dans l'usine i,

$a_{ij}$ le cout de production pour la glace j dans l'usine i.

Nous posons maintenant le problème d'optimisation.

$$\begin{align*} \min_x\ & B^{\intercal}x + E[Q(x)]\\ \mbox{s.t. } & x \geq 0 \\ & B^{\intercal}x \leq 120 \\ & e^{\intercal}x \geq 12 \end{align*}$$

où le problème de 2e étape est
$$\begin{align*} Q(x) = \min_y\ & e^{\intercal}A_{mat} \circ y_{mat} e \\ \mbox{s.t. } & y_{mat}e \leq x \\ & y_{mat}^{\intercal}e \geq (\xi, 3, 2)^{\intercal} \end{align*}$$

ici $\circ$ est le produit d'Hadamard et les opérations d'inégalités ce font éléments par éléments entre deux vecteurs. $e$ est un vecteur colonne unitaire associée à la matrice le précédent.

## Problème avec décision vectorielle

Par contre, dans notre théorie du cours, nous avons vue que la décision est toujours prises comme étant un vecteur. Celà est du à la formulation des W, h et T que nous aurons besoin dans le calcul des coupes. Dans la formulation précédente, nous avons décider intuitivement d'utiliser une formulation avec des matrices de décision. Cela est du à la facilité de pouvoir associer $y_{ij}$ à la glace j dans l'usine i. Nous récrivons notre problème cette fois sous forme vectorielle.

$$ x = (x_1, x_2, x_3, x_4)^{\intercal}   \\
B = (10, 7, 16, 6)^{\intercal}  \\
y_{vec}= (y_1, y_2, y_3, y_4, y_5, y_6, y_7, y_8, y_9, y_{10}, y_{11}, y_{12})^{\intercal} \\
A_{vec} = (40, 24 , 4, 45 , 27 , 4.5 , 32 , 19.2 , 3.2 , 55 , 33 , 5.5)^{\intercal} $$

où $x_i$ représente la capacité de production de l'usine i,

B le vector des coûts de production,

$y_1, y_2, y_3$ est la production de la saveur 1, 2 et 3 dans l'usine 1 respectivement,

$y_4, y_5, y_6$ est la production de la saveur 1, 2 et 3 dans l'usine 2,

et ainsi de suite.

Nous pouvons écrire la relation suivante

$y_{j}$ la production de la saveur $\mod(j-1, 3) + 1$ dans l'usine $\lceil \frac{j-1}{3} \rceil + 1$

$a_{j}$ le cout de production des saveurs définie de manière identique à y.

Nous posons maintenant le problème d'optimisation.

\begin{align*}
    \min_x\ & B^{\intercal} x + E[Q(x)]\\
    \mbox{s.t. } & x \geq 0 \\
     & B^{\intercal}x \leq 120 \\
     & e^{\intercal}x \geq 12
\end{align*}

\begin{align*}
Q(x) &= A_{vec}^{\intercal}y_{vec}\\
\mbox{s.t. } & Wy_{vec}= h(\xi) + Tx
\end{align*}

où

$W = \begin{pmatrix}
1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 \\
1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 \\
0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1
\end{pmatrix}$

$ h(\xi) = \begin{pmatrix} 0 & 0 & 0 & 0 & \xi & 3 & 2 \end{pmatrix} ^{\intercal}$

$ T = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{pmatrix}$

In [2]:
# Définition des variables du problème
scenarios = 3
p = [0.3, 0.4, 0.3]
ξ = [3, 5, 7]
B = [10 7 16 6]
A_mat = [40 24 4; 45 27 4.5; 32 19.2 3.2; 55 33 5.5]
A_vec = [40 24 4 45 27 4.5 32 19.2 3.2 55 33 5.5]
W = [1 1 1 0 0 0 0 0 0 0 0 0;
     0 0 0 1 1 1 0 0 0 0 0 0;
     0 0 0 0 0 0 1 1 1 0 0 0;
     0 0 0 0 0 0 0 0 0 1 1 1;
     1 0 0 1 0 0 1 0 0 1 0 0;
     0 1 0 0 1 0 0 1 0 0 1 0;
     0 0 1 0 0 1 0 0 1 0 0 1]
T = [1 0 0 0;
     0 1 0 0;
     0 0 1 0;
     0 0 0 1;
     0 0 0 0;
     0 0 0 0;
     0 0 0 0]
#theta = -Inf

7×4 Array{Int64,2}:
 1  0  0  0
 0  1  0  0
 0  0  1  0
 0  0  0  1
 0  0  0  0
 0  0  0  0
 0  0  0  0

In [3]:
function master(theta)
    master = Model(solver=ClpSolver())
    
    @variable(master, x[1:4] >= 0)
    
    @constraint(master, sum(x[i]*B[i] for i=1:4) <= 120)
    @constraint(master, sum(x[i] for i = 1:4) >= 12)
    
    @objective(master, Min, sum(x[i]*B[i] for i=1:4) + theta)
    
    status = solve(master)
    
    return master, status, getvalue(x)
end

master (generic function with 1 method)

Testons notre problème master sans l'influence de theta

In [4]:
test, stat, x = master(-Inf)
println(x)

[0.0, 0.0, 0.0, 12.0]


Nous écrivons maintenant notre fonction pour générer et cacluler notre problème de deuxième étape. Avec JuMP, il est facile d'écire un modèle avec une matrice de variables de décision comme nous pouvons constater dans le code suivant.

In [5]:
function secondstage_mat(x::Vector, ξ::Int)
    m = Model(solver=ClpSolver())    

    @variable(m, y[1:4,1:3] >= 0)
    
    @constraintref artCons[1:7]
    artCons[1] = @constraint(m, sum(y[1,j] for j=1:3) <= x[1])
    artCons[2] = @constraint(m, sum(y[2,j] for j=1:3) <= x[2])
    artCons[3] = @constraint(m, sum(y[3,j] for j=1:3) <= x[3])
    artCons[4] = @constraint(m, sum(y[4,j] for j=1:3) <= x[4])
    artCons[5] = @constraint(m, sum(y[i,1] for i=1:4) >= ξ)
    artCons[6] = @constraint(m, sum(y[i,2] for i=1:4) >= 3)
    artCons[7] = @constraint(m, sum(y[i,3] for i=1:4) >= 2)
        
    @objective(m, Min, sum(A_mat[i,j]*y[i,j] for i=1:4, j=1:3))
    
    status = solve(m)
    
    dual = getdual(artCons)
    
    
    return m, status, getvalue(y), dual
end

secondstage_mat (generic function with 1 method)

Comparons la formulation avec matrice de décision à une formulation de vecteur de décision et observons ce que JuMP va nous donner pour le vecteur dual.

In [6]:
function secondstage_vec(x::Vector, ξ::Int)
    m = Model(solver=ClpSolver())    

    
    @variable(m, y[1:12] >= 0)
    
    @constraintref artCons[1:7]
    artCons[1] = @constraint(m, sum(y[j] for j=1:3) <= x[1])
    artCons[2] = @constraint(m, sum(y[j] for j=4:6) <= x[2])
    artCons[3] = @constraint(m, sum(y[j] for j=7:9) <= x[3])
    artCons[4] = @constraint(m, sum(y[j] for j=10:12) <= x[4])
    artCons[5] = @constraint(m, y[1] + y[4] + y[7] + y[10] >= ξ)
    artCons[6] = @constraint(m, y[2] + y[5] + y[8] + y[11] >= 3)
    artCons[7] = @constraint(m, y[3] + y[6] + y[9] + y[12] >= 2)
       
    @objective(m, Min, sum(A_vec*y))
    
    status = solve(m)
    
    dual = getdual(artCons)  
    
    return m, status, getvalue(y), dual
end

secondstage_vec (generic function with 1 method)

In [7]:
martificial_mat, status_mat, value_y_mat, dual_mat = secondstage_mat([0,0,0,12], 3)

(Minimization problem with:
 * 7 linear constraints
 * 12 variables
Solver is ClpMathProg, :Optimal, [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 3.0 3.0 2.0], [-15.0, -10.0, -23.0, 0.0, 55.0, 33.0, 5.5])

In [8]:
martificial_vec, status_vec, value_y_vec, dual_vec = secondstage_vec([0,0,0,12], 3)

(Minimization problem with:
 * 7 linear constraints
 * 12 variables
Solver is ClpMathProg, :Optimal, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 2.0], [-15.0, -10.0, -23.0, 0.0, 55.0, 33.0, 5.5])

In [9]:
print(martificial_mat)

Min 40 y[1,1] + 24 y[1,2] + 4 y[1,3] + 45 y[2,1] + 27 y[2,2] + 4.5 y[2,3] + 32 y[3,1] + 19.2 y[3,2] + 3.2 y[3,3] + 55 y[4,1] + 33 y[4,2] + 5.5 y[4,3]
Subject to
 y[1,1] + y[1,2] + y[1,3] ≤ 0
 y[2,1] + y[2,2] + y[2,3] ≤ 0
 y[3,1] + y[3,2] + y[3,3] ≤ 0
 y[4,1] + y[4,2] + y[4,3] ≤ 12
 y[1,1] + y[2,1] + y[3,1] + y[4,1] ≥ 3
 y[1,2] + y[2,2] + y[3,2] + y[4,2] ≥ 3
 y[1,3] + y[2,3] + y[3,3] + y[4,3] ≥ 2
 y[i,j] ≥ 0 ∀ i ∈ {1,2,3,4}, j ∈ {1,2,3}


In [10]:
print(dual_mat)

[-15.0, -10.0, -23.0, 0.0, 55.0, 33.0, 5.5]

In [11]:
print(martificial_vec)

Min 40 y[1] + 24 y[2] + 4 y[3] + 45 y[4] + 27 y[5] + 4.5 y[6] + 32 y[7] + 19.2 y[8] + 3.2 y[9] + 55 y[10] + 33 y[11] + 5.5 y[12]
Subject to
 y[1] + y[2] + y[3] ≤ 0
 y[4] + y[5] + y[6] ≤ 0
 y[7] + y[8] + y[9] ≤ 0
 y[10] + y[11] + y[12] ≤ 12
 y[1] + y[4] + y[7] + y[10] ≥ 3
 y[2] + y[5] + y[8] + y[11] ≥ 3
 y[3] + y[6] + y[9] + y[12] ≥ 2
 y[i] ≥ 0 ∀ i ∈ {1,2,…,11,12}


In [12]:
print(dual_vec)

[-15.0, -10.0, -23.0, 0.0, 55.0, 33.0, 5.5]

Nous observons que Julia donne exactement les mêmes vecteurs dual que le problème soit avec y vectorielle ou matricielle. Nous assumons que ceci est une fonctionnalité de JuMP. Nous allons donc continuer de travailler avec la formulation matricielle du problème puisque celle-ci est plus jolie. Nous devons juste nous assurer de bien écrire nos vecteurs W, h et T selon la formulation des contraintes.

In [85]:
function secondstage_feasability(x::Vector, ξ::Int)
    m = Model(solver = ClpSolver())

    @variable(m, y[1:4,1:3] >= 0)
    @variable(m, splus[1:7] >= 0)
    @variable(m, sminus[1:7] >= 0)

    @constraintref artCons[1:7]
    artCons[1] = @constraint(m, sum(y[1,j] for j=1:3) + splus[1] - sminus[1] <= x[1])
    artCons[2] = @constraint(m, sum(y[2,j] for j=1:3) + splus[2] - sminus[2] <= x[2])
    artCons[3] = @constraint(m, sum(y[3,j] for j=1:3) + splus[3] - sminus[3] <= x[3])
    artCons[4] = @constraint(m, sum(y[4,j] for j=1:3) + splus[4] - sminus[4] <= x[4])
    artCons[5] = @constraint(m, sum(y[i,1] for i=1:4) + splus[5] - sminus[5] >= ξ)
    artCons[6] = @constraint(m, sum(y[i,2] for i=1:4) + splus[6] - sminus[6] >= 3)
    artCons[7] = @constraint(m, sum(y[i,3] for i=1:4) + splus[7] - sminus[7] >= 2)
    
    @objective(m, Min, sum(splus[i] for i=1:7)+sum(sminus[i] for i=1:7))
    
    status = solve(m)

    dual = getdual(artCons)

    return m, status, getobjectivevalue(m), dual
end

secondstage_feasability (generic function with 1 method)

In [86]:
function optimalcut(x::Vector, ξ::Int, p::Float64, dual::Vector)
    h =  [0; 0; 0; 0; ξ; 3; 2];
    T =  [-x[1] 0 0 0; 0 -x[2] 0 0; 0 0 -x[3] 0; 0 0 0 -x[4]; 0 0 0 0; 0 0 0 0; 0 0 0 0]   
    e = p * dual'* h
    E = p * dual'*T
    return E, e
end

optimalcut (generic function with 1 method)

In [203]:
optimalcut([0,0,0,12], 3, 0.5, dual)

([0.0 0.0 0.0 0.0], 137.5)

In [83]:
function feascut(x::Vector, ξ::Int, dual::Vector)
    h =  [ 0; 0; 0; 0; ξ; 3; 2]
    T =  [-x[1] 0 0 0; 0 -x[2] 0 0; 0 0 -x[3] 0; 0 0 0 -x[4]; 0 0 0 0; 0 0 0 0; 0 0 0 0] 
    D = dual'*T
    d = dual'*h
    return D, d
end

feascut (generic function with 1 method)

In [156]:
K = 0
L = 0
stop = 0
p = [0.3 0.4 0.3]
ξ = [3 5 7]
optconstraintcounter = 1
feasconstraintcounter = 1
interation = 0

mas = Model(solver=ClpSolver())

@variable(mas, x[1:4] >= 0)

# malheureusement, nous ne pouvons pas utiliser la valeur de -Inf sinon le probleme est non borne
@variable(mas, theta >= -99999999999999) 

@constraint(mas, sum(x[i]*B[i] for i=1:4) <= 120)
@constraint(mas, sum(x[i] for i = 1:4) >= 12)
    
@objective(mas, Min, sum(x[i]*B[i] for i=1:4) + theta)
    
status = solve(mas)
Q_rond = getobjectivevalue(mas)
a = getvalue(x)

4-element Array{Float64,1}:
  0.0
  0.0
  0.0
 12.0

In [165]:
@printf("%f\n",Q_rond)
if (status == :Infeasible)
    stop = 1
    println("Problem is infeasible.")
end

457.000000


In [166]:
#while (stop == 0)
    x_mod = getvalue(x)
    ### On fait une coupe de faisabiliter si necessaire et on recommence jusqu'a temps qu'on ne peut plus.

    obj = 0
    for i = 1:3
        m, status, temp_obj, dual = secondstage_feasability(x_mod, ξ[i])
        
        #println(tmp_d)
        #println(tmp_D)
        obj += temp_obj
        
        if (temp_obj > 0)
            tmp_D, tmp_d = feascut(x_mod, realisation[i], dual)
            @constraint(mas, cut[constraintcounter], sum(big_sig[i]*x[i] for i = 1:4) >= small_sig)
            feasconstraintcounter += 1
        end
    end

In [167]:
print(obj)

0.0

In [168]:
    ### Coupe d'optimalité
    if(obj == 0)  

LoadError: [91msyntax: incomplete: premature end of input[39m

In [169]:
        # il n'y a pas eu de coupe de faisabilité puisque tout les valuers objective était nul
        # on commence nos coupes d'optimalitées
        E = [0 0 0 0]
        e = 0
        #on accumule E et e pour tous les realisation (pour faire la sommation)
        for i=1:3
            m, status, y, dual = secondstage_mat(x_mod, ξ[i])
            temp_E, temp_e = optimalcut(x_mod, ξ[i], p[i], dual)
            E += temp_E
            e += temp_e    
        end
        #on calcule notre q_rond
        Q_rond = e - dot(E, x_mod)
    

385.0

In [201]:
m, status, y, dual = secondstage_vec(x_mod, ξ[3])
dual

7-element Array{Float64,1}:
 -15.0
 -10.0
 -23.0
   0.0
  55.0
  33.0
   5.5

In [176]:
        #si il est plus petit que la valeur objective, on arrete
        if (Q_rond < getobjectivevalue(mas))
            stop = 1
            println("L-shaped solved")
        else
            #sinon, on rajoute une coupe
            @constraint(mas, sum(E[i]*x[i] for i = 1:4) + theta >= e)
            optconstraintcounter += 1
        end
    
        #dans le cas ou le stop n'est pas egal a un, on resous notre nouveau probleme avec la nouvelle contrainte,
        #s'il devient infaisaible, on arrete
        if (stop != 1)
            status = solve(mas)
            Q_rond = getobjectivevalue(mas)
            if(status == :Infeasible)
                stop = 1
                println("Problem is infeasible.")
            end
            println(Q_rond)
        end 

L-shaped solved


In [177]:
getvalue(theta)

385.0

In [179]:
getobjectivevalue(mas)

457.0

In [99]:
    end    
    iteration += 1
    if (iteration > 1000)
        println("Exceeded maximum number of iterations")
        break
    end
end

4-element Array{Float64,1}:
  0.0
  0.0
  0.0
 12.0